In [147]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [148]:
# read data from file
file_path = './data/heart.csv'
data = pd.read_csv(file_path, header=0)


# shuffle data
data = data.sample(frac=1)


# split data into x_train and y_train
x_train = data.iloc[:, :-1].values
y_train = data.iloc[:, -1].values


In [149]:
# split data to train and test
test_size = 0.2
test_count = int(len(x_train) * test_size)

# split data
x_test = x_train[:test_count]
y_test = y_train[:test_count]
x_train = x_train[test_count:]
y_train = y_train[test_count:]

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(243, 13) (243,) (60, 13) (60,)


In [150]:
# normalize data
norm_l = tf.keras.layers.experimental.preprocessing.Normalization()
norm_l.adapt(x_train)
original_x = x_train
x_train = norm_l(x_train)
x_test = norm_l(x_test)
print(x_train[0:3])

tf.Tensor(
[[ 0.38011557 -1.4966629   1.0601641  -0.6516637   1.841484   -0.41020632
   0.91923875  0.9881739  -0.7136643  -0.91368604  0.99059266 -0.7328202
  -0.4858548 ]
 [ 1.3627542  -1.4966629   1.0601641  -0.9363712   6.2554016  -0.41020632
  -0.9899494   0.4788022  -0.7136643   0.41630706 -0.63585377 -0.7328202
   1.109587  ]
 [ 1.035208   -1.4966629  -0.89150155 -0.08224873  1.1123992  -0.41020632
   0.91923875 -1.1342081  -0.7136643   0.74880534 -0.63585377  1.2793305
  -0.4858548 ]], shape=(3, 13), dtype=float32)


In [151]:
# tile data
x_train = tf.tile(x_train, (100,1))
x_test = tf.tile(x_test, (100,1))
y_train = tf.tile(y_train, [100])
y_test = tf.tile(y_test, [100])
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(24300, 13) (24300,) (6000, 13) (6000,)


In [152]:
# build model
model = Sequential()
model.add(Dense(10, input_shape=(x_train.shape[1] ,), activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_37 (Dense)            (None, 10)                140       
                                                                 
 dense_38 (Dense)            (None, 10)                110       
                                                                 
 dense_39 (Dense)            (None, 1)                 11        
                                                                 
Total params: 261
Trainable params: 261
Non-trainable params: 0
_________________________________________________________________


In [153]:
# compile model
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)

model.fit(
    x_train,y_train,            
    epochs=10,
)

Epoch 1/10
760/760 [==============================] - 2s 1ms/step - loss: 0.2975
Epoch 2/10
760/760 [==============================] - 1s 2ms/step - loss: 0.0503
Epoch 3/10
760/760 [==============================] - 1s 1ms/step - loss: 0.0047
Epoch 4/10
760/760 [==============================] - 1s 1ms/step - loss: 0.0014
Epoch 5/10
760/760 [==============================] - 1s 1ms/step - loss: 6.1811e-04
Epoch 6/10
760/760 [==============================] - 1s 2ms/step - loss: 3.1799e-04
Epoch 7/10
760/760 [==============================] - 1s 2ms/step - loss: 1.7985e-04
Epoch 8/10
760/760 [==============================] - 1s 1ms/step - loss: 1.0732e-04
Epoch 9/10
760/760 [==============================] - 1s 1ms/step - loss: 6.6722e-05
Epoch 10/10
760/760 [==============================] - 1s 1ms/step - loss: 4.2172e-05


In [154]:
# predict
def predict(x):
    return tf.reshape(tf.cast(model.predict(x) > 0.5, tf.int32), [-1])

yte_pred = predict(x_test)
ytr_pred = predict(x_train)

# accuracy
train_accuracy = tf.reduce_mean(tf.cast(tf.equal(ytr_pred, tf.cast(y_train, tf.int32)), tf.float32))
test_accuracy = tf.reduce_mean(tf.cast(tf.equal(yte_pred, tf.cast(y_test, tf.int32)), tf.float32))

print("Train accuracy:", train_accuracy.numpy())
print("Test accuracy:", test_accuracy.numpy())


760/760 [==============================] - 1s 1ms/step
Train accuracy: 1.0
Test accuracy: 0.8
